# Prework

## Functions and libraries

In [8]:
import os
# PATH = "/Users/luanagiusto/TP-1-ML"  # Cambia esto si tu path es diferente
PATH = "C:/Users/julia/OneDrive/Escritorio/Archivos/Capacitación/Maestría/03. Machine Learning/TP"

In [16]:
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
import gc
# Nota: Antes de ejecutar este notebook, instala los requisitos con:
# !pip install -r requirements.txt

In [17]:
def data_profiling(df, output_file):
    # Opciones para que sea liviano
    profile = ProfileReport(
        df.sample(20000, random_state=42) if len(df) > 20000 else df,
        title=output_file,
        minimal=True,         # desactiva análisis costosos
        explorative=True      # agrega secciones útiles
    )

    profile.to_file(output_file)  # <-- abre este HTML en el navegador

In [18]:
# Funcion para mostrar un resumen del dataframe
def df_info_summary(df: pd.DataFrame):
    total = len(df)
    non_null = df.notnull().sum()
    nulls = df.isnull().sum()
    dtypes = df.dtypes
    
    resumen = pd.DataFrame({
        "Non-Null Count": non_null,
        "Null Count": nulls,
        "% Null": (nulls / total * 100).round(2),
        "Dtype": dtypes
    })
    print(resumen)

In [19]:
def resumir_por_id(df, id_col='ID', excluir_cols=None, verbose=False, nombre_conteo='n_registros'):
    """
    Sumariza un DataFrame agrupando por una columna ID.
    Calcula métricas estadísticas básicas para columnas numéricas,
    excluyendo las que se indiquen. Incluye conteo total de registros por ID.

    Parámetros:
    - df: DataFrame de entrada con múltiples registros por ID.
    - id_col: nombre de la columna que identifica cada entidad única.
    - excluir_cols: lista de columnas a excluir del resumen (opcional).
    - verbose: si True, imprime columnas incluidas y excluidas.
    - nombre_conteo: nombre de la columna que indica cantidad de registros por ID.

    Retorna:
    - DataFrame con una fila por ID y métricas estadísticas por columna.
    """
    if excluir_cols is None:
        excluir_cols = []

    excluir_set = set(excluir_cols)
    if id_col in excluir_set:
        excluir_set.remove(id_col)

    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    cols_a_resumir = [col for col in numeric_cols if col not in excluir_set and col != id_col]

    if verbose:
        print(f"Columnas excluidas: {sorted(excluir_set)}")
        print(f"Columnas resumidas: {sorted(cols_a_resumir)}")

    # Agregaciones estadísticas
    agg_funcs = ['mean', 'min', 'max', 'median', 'sum']
    agg_dict = {col: agg_funcs for col in cols_a_resumir}

    # Agregar conteo de registros por ID
    df[nombre_conteo] = 1
    agg_dict[nombre_conteo] = ['count']

    resumen = df.groupby(id_col).agg(agg_dict)
    resumen.columns = [f"{col}_{stat}" for col, stat in resumen.columns]
    resumen = resumen.reset_index()

    return resumen

## Data import and overview

In [20]:
# PATH = "/Users/luanagiusto/TP-1-ML"  # Cambia esto si tu path es diferente
PATH = "C:/Users/julia/OneDrive/Escritorio/Archivos/Capacitación/Maestría/03. Machine Learning/TP"
train_df = pd.read_parquet(os.path.join(PATH, "train.parquet"), engine='fastparquet')

In [21]:
df_info_summary(train_df)

                                          Non-Null Count  Null Count  % Null    Dtype
SK_ID_CURR                                        307511           0    0.00    int64
TARGET                                            307511           0    0.00    int64
NAME_CONTRACT_TYPE                                307511           0    0.00   object
CODE_GENDER                                       307511           0    0.00   object
FLAG_OWN_CAR                                      307511           0    0.00   object
FLAG_OWN_REALTY                                   307511           0    0.00   object
CNT_CHILDREN                                      307511           0    0.00    int64
AMT_INCOME_TOTAL                                  307511           0    0.00  float64
AMT_CREDIT                                        307511           0    0.00  float64
AMT_ANNUITY                                       307499          12    0.00  float64
AMT_GOODS_PRICE                                   3072

In [3]:
# La librería fastai tiene varias funcionalidad para preprocesar datasets. 
# En particular para poder usar un Random Forests de ScikitLearn necesitamos que todas las variables sean numéricas y no tener valores faltantas.
# En particular vamos a usar dos preprocesamientos: Categorify y FillMissing. Por ahora solo definimos los preprocesamientos y luego se aplican.
procs = [Categorify, FillMissing]

In [4]:
# Además del preprocesamiento, vamos a dividir nuestro dataset en train y validation. 
# - dep_var='TARGET': excluye esta columna del análisis porque es la variable que se quiere predecir.
# - preproc_names=procs: aplica transformaciones como Categorify, FillMissing, que están definidas en procs

cont,cat = cont_cat_split(train_df, 1, dep_var='TARGET')

In [5]:
# Índices del DataFrame completo
idxs = list(range(len(train_df)))

# Partición aleatoria 80/20
train_idx, valid_idx = train_test_split(idxs, test_size=0.2, random_state=42)

# Adaptación para TabularPandas
splits = (list(train_idx), list(valid_idx))


In [6]:
# train_set, valid_set = train_test_split(
#     train_df,
#     test_size=0.2,
#     random_state=42
# )

In [7]:
to = TabularPandas(train_df, procs, cat, cont, y_names='TARGET', splits=splits)

In [8]:
len(to.train),len(to.valid)

(246008, 61503)

In [9]:
# Una vez hecho el preprocesamiento, se puede ver que los valores del dataframe son todos numéricos.

to.items.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BB_SK_ID_CURR,BB_CREDIT_ACTIVE_mean,BB_CREDIT_ACTIVE_min,BB_CREDIT_ACTIVE_max,BB_CREDIT_ACTIVE_median,BB_CREDIT_ACTIVE_sum,BB_CREDIT_CURRENCY_mean,BB_CREDIT_CURRENCY_min,BB_CREDIT_CURRENCY_max,BB_CREDIT_CURRENCY_median,BB_CREDIT_CURRENCY_sum,BB_CREDIT_TYPE_mean,BB_CREDIT_TYPE_min,BB_CREDIT_TYPE_max,BB_CREDIT_TYPE_median,BB_CREDIT_TYPE_sum,BB_AMT_ANNUITY_na_mean,BB_AMT_ANNUITY_na_min,BB_AMT_ANNUITY_na_max,BB_AMT_ANNUITY_na_median,BB_AMT_ANNUITY_na_sum,BB_AMT_CREDIT_MAX_OVERDUE_na_mean,BB_AMT_CREDIT_MAX_OVERDUE_na_min,BB_AMT_CREDIT_MAX_OVERDUE_na_max,BB_AMT_CREDIT_MAX_OVERDUE_na_median,BB_AMT_CREDIT_MAX_OVERDUE_na_sum,BB_AMT_CREDIT_SUM_na_mean,BB_AMT_CREDIT_SUM_na_min,BB_AMT_CREDIT_SUM_na_max,BB_AMT_CREDIT_SUM_na_median,BB_AMT_CREDIT_SUM_na_sum,BB_AMT_CREDIT_SUM_DEBT_na_mean,BB_AMT_CREDIT_SUM_DEBT_na_min,BB_AMT_CREDIT_SUM_DEBT_na_max,BB_AMT_CREDIT_SUM_DEBT_na_median,BB_AMT_CREDIT_SUM_DEBT_na_sum,BB_AMT_CREDIT_SUM_LIMIT_na_mean,BB_AMT_CREDIT_SUM_LIMIT_na_min,BB_AMT_CREDIT_SUM_LIMIT_na_max,BB_AMT_CREDIT_SUM_LIMIT_na_median,BB_AMT_CREDIT_SUM_LIMIT_na_sum,BB_DAYS_CREDIT_ENDDATE_na_mean,BB_DAYS_CREDIT_ENDDATE_na_min,BB_DAYS_CREDIT_ENDDATE_na_max,BB_DAYS_CREDIT_ENDDATE_na_median,BB_DAYS_CREDIT_ENDDATE_na_sum,BB_DAYS_ENDDATE_FACT_na_mean,BB_DAYS_ENDDATE_FACT_na_min,BB_DAYS_ENDDATE_FACT_na_max,BB_DAYS_ENDDATE_FACT_na_median,BB_DAYS_ENDDATE_FACT_na_sum,BB_AMT_ANNUITY_mean,BB_AMT_ANNUITY_min,BB_AMT_ANNUITY_max,BB_AMT_ANNUITY_median,BB_AMT_ANNUITY_sum,BB_AMT_CREDIT_MAX_OVERDUE_mean,BB_AMT_CREDIT_MAX_OVERDUE_min,BB_AMT_CREDIT_MAX_OVERDUE_max,BB_AMT_CREDIT_MAX_OVERDUE_median,BB_AMT_CREDIT_MAX_OVERDUE_sum,BB_AMT_CREDIT_SUM_mean,BB_AMT_CREDIT_SUM_min,BB_AMT_CREDIT_SUM_max,BB_AMT_CREDIT_SUM_median,BB_AMT_CREDIT_SUM_sum,BB_AMT_CREDIT_SUM_DEBT_mean,BB_AMT_CREDIT_SUM_DEBT_min,BB_AMT_CREDI

In [10]:
to.items.describe(include='all')  # Para ver estadísticas generales

SK_ID_CURR         TARGET  NAME_CONTRACT_TYPE    CODE_GENDER   FLAG_OWN_CAR  FLAG_OWN_REALTY   CNT_CHILDREN  AMT_INCOME_TOTAL    AMT_CREDIT    AMT_ANNUITY  AMT_GOODS_PRICE  NAME_TYPE_SUITE  NAME_INCOME_TYPE  NAME_EDUCATION_TYPE  NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  REGION_POPULATION_RELATIVE     DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  DAYS_ID_PUBLISH    OWN_CAR_AGE     FLAG_MOBIL  FLAG_EMP_PHONE  FLAG_WORK_PHONE  FLAG_CONT_MOBILE     FLAG_PHONE     FLAG_EMAIL  OCCUPATION_TYPE  CNT_FAM_MEMBERS  REGION_RATING_CLIENT  REGION_RATING_CLIENT_W_CITY  WEEKDAY_APPR_PROCESS_START  HOUR_APPR_PROCESS_START  REG_REGION_NOT_LIVE_REGION  REG_REGION_NOT_WORK_REGION  LIVE_REGION_NOT_WORK_REGION  REG_CITY_NOT_LIVE_CITY  REG_CITY_NOT_WORK_CITY  LIVE_CITY_NOT_WORK_CITY  ORGANIZATION_TYPE   EXT_SOURCE_1  EXT_SOURCE_2   EXT_SOURCE_3  APARTMENTS_AVG  BASEMENTAREA_AVG  YEARS_BEGINEXPLUATATION_AVG  YEARS_BUILD_AVG  COMMONAREA_AVG  ELEVATORS_AVG  ENTRANCES_AVG  FLOORSMAX_AVG  FLOORSMIN_AVG   LANDAREA_AVG  LIVINGAPARTMENTS_AVG  LIVINGAREA_AVG  NONLIVINGAPARTMENTS_AVG  NONLIVINGAREA_AVG  APARTMENTS_MODE  BASEMENTAREA_MODE  YEARS_BEGINEXPLUATATION_MODE  YEARS_BUILD_MODE  COMMONAREA_MODE  ELEVATORS_MODE  ENTRANCES_MODE  FLOORSMAX_MODE  FLOORSMIN_MODE  LANDAREA_MODE  LIVINGAPARTMENTS_MODE  LIVINGAREA_MODE  NONLIVINGAPARTMENTS_MODE  NONLIVINGAREA_MODE  APARTMENTS_MEDI  BASEMENTAREA_MEDI  YEARS_BEGINEXPLUATATION_MEDI  YEARS_BUILD_MEDI  COMMONAREA_MEDI  ELEVATORS_MEDI  ENTRANCES_MEDI  FLOORSMAX_MEDI  FLOORSMIN_MEDI  LANDAREA_MEDI  LIVINGAPARTMENTS_MEDI  LIVINGAREA_MEDI  NONLIVINGAPARTMENTS_MEDI  NONLIVINGAREA_MEDI  FONDKAPREMONT_MODE  HOUSETYPE_MODE  TOTALAREA_MODE  WALLSMATERIAL_MODE  EMERGENCYSTATE_MODE  OBS_30_CNT_SOCIAL_CIRCLE  DEF_30_CNT_SOCIAL_CIRCLE  OBS_60_CNT_SOCIAL_CIRCLE  DEF_60_CNT_SOCIAL_CIRCLE  DAYS_LAST_PHONE_CHANGE  FLAG_DOCUMENT_2  FLAG_DOCUMENT_3  FLAG_DOCUMENT_4  FLAG_DOCUMENT_5  FLAG_DOCUMENT_6  FLAG_DOCUMENT_7  FLAG_DOCUMENT_8  FLAG_DOCUMENT_9  FLAG_DOCUMENT_10  FLAG_DOCUMENT_11  FLAG_DOCUMENT_12  FLAG_DOCUMENT_13  FLAG_DOCUMENT_14  FLAG_DOCUMENT_15  FLAG_DOCUMENT_16  FLAG_DOCUMENT_17  FLAG_DOCUMENT_18  FLAG_DOCUMENT_19  FLAG_DOCUMENT_20  FLAG_DOCUMENT_21  AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  BB_SK_ID_CURR  BB_CREDIT_ACTIVE_mean  BB_CREDIT_ACTIVE_min  BB_CREDIT_ACTIVE_max  BB_CREDIT_ACTIVE_median  BB_CREDIT_ACTIVE_sum  BB_CREDIT_CURRENCY_mean  BB_CREDIT_CURRENCY_min  BB_CREDIT_CURRENCY_max  BB_CREDIT_CURRENCY_median  BB_CREDIT_CURRENCY_sum  BB_CREDIT_TYPE_mean  BB_CREDIT_TYPE_min  BB_CREDIT_TYPE_max  BB_CREDIT_TYPE_median  BB_CREDIT_TYPE_sum  BB_AMT_ANNUITY_na_mean  BB_AMT_ANNUITY_na_min  BB_AMT_ANNUITY_na_max  BB_AMT_ANNUITY_na_median  BB_AMT_ANNUITY_na_sum  BB_AMT_CREDIT_MAX_OVERDUE_na_mean  BB_AMT_CREDIT_MAX_OVERDUE_na_min  BB_AMT_CREDIT_MAX_OVERDUE_na_max  BB_AMT_CREDIT_MAX_OVERDUE_na_median  BB_AMT_CREDIT_MAX_OVERDUE_na_sum  BB_AMT_CREDIT_SUM_na_mean  BB_AMT_CREDIT_SUM_na_min  BB_AMT_CREDIT_SUM_na_max  BB_AMT_CREDIT_SUM_na_median  BB_AMT_CREDIT_SUM_na_sum  BB_AMT_CREDIT_SUM_DEBT_na_mean  BB_AMT_CREDIT_SUM_DEBT_na_min  BB_AMT_CREDIT_SUM_DEBT_na_max  BB_AMT_CREDIT_SUM_DEBT_na_median  BB_AMT_CREDIT_SUM_DEBT_na_sum  BB_AMT_CREDIT_SUM_LIMIT_na_mean  BB_AMT_CREDIT_SUM_LIMIT_na_min  BB_AMT_CREDIT_SUM_LIMIT_na_max  BB_AMT_CREDIT_SUM_LIMIT_na_median  BB_AMT_CREDIT_SUM_LIMIT_na_sum  BB_DAYS_CREDIT_ENDDATE_na_mean  BB_DAYS_CREDIT_ENDDATE_na_min  BB_DAYS_CREDIT_ENDDATE_na_max  BB_DAYS_CREDIT_ENDDATE_na_median  BB_DAYS_CREDIT_ENDDATE_na_sum  BB_DAYS_ENDDATE_FACT_na_mean  BB_DAYS_ENDDATE_FACT_na_min  BB_DAYS_ENDDATE_FACT_na_max  BB_DAYS_ENDDATE_FACT_na_median  BB_DAYS_ENDDATE_FACT_na_sum  BB_AMT_ANNUITY_mean  BB_AMT_ANNUITY_min  BB_AMT_ANNUITY_max  BB_AMT_ANNUITY_median  BB_AMT_ANNUITY_sum  BB_AMT_CREDIT_MAX_OVERDUE_mean  BB_AMT_CREDIT_MAX_OVERDUE_min  BB_AMT_CREDIT_MAX_OVERDUE_max  BB_AMT_CREDIT_MAX_OVERDUE_median

In [11]:
to.train.xs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246008 entries, 123473 to 121958
Columns: 1452 entries, NAME_CONTRACT_TYPE to CCB_credit_card_balance_records_count
dtypes: float32(700), int16(2), int32(2), int8(748)
memory usage: 837.1 MB


In [12]:
to.train.xs.dtypes


NAME_CONTRACT_TYPE                                int8
CODE_GENDER                                       int8
FLAG_OWN_CAR                                      int8
FLAG_OWN_REALTY                                   int8
NAME_TYPE_SUITE                                   int8
NAME_INCOME_TYPE                                  int8
NAME_EDUCATION_TYPE                               int8
NAME_FAMILY_STATUS                                int8
NAME_HOUSING_TYPE                                 int8
OCCUPATION_TYPE                                   int8
WEEKDAY_APPR_PROCESS_START                        int8
ORGANIZATION_TYPE                                 int8
FONDKAPREMONT_MODE                                int8
HOUSETYPE_MODE                                    int8
WALLSMATERIAL_MODE                                int8
EMERGENCYSTATE_MODE                               int8
AMT_ANNUITY_na                                    int8
AMT_GOODS_PRICE_na                                int8
OWN_CAR_AG

In [13]:
# Dado que el preprocesamiento puede tardar, guardamos el objeto en memoria para futuro uso
save_pickle(os.path.join(PATH, 'prework-tabular-object.pkl'), to)